Solution of the exercises
=========================

Solution of exercise 1
----------------------

Here's the solution for the IPT Solver written as a class:

In [ ]:
from pytriqs.gf.local import *

class IPTSolver:

    def __init__(self, beta):

        self.beta = beta

        # Matsubara frequency
        # self.g0 will be set by the user after initialization
        self.g = GfImFreq(indices=[0], beta=beta, name='0')
        self.g0 = self.g.copy()
        self.sigma = self.g.copy()
        
        # Imaginary time
        self.g0t = GfImTime(indices=[0], beta = self.beta, name='G0t')
        self.sigmat = self.g0t.copy()
        
    def solve(self, U,it):

        self.g0t << InverseFourier(self.g0)
        self.sigmat << (U**2) * self.g0t * self.g0t * self.g0t
        self.sigma << Fourier(self.sigmat)
        self.sigma << (U/2.0 if it>8 else 0) + self.sigma
        
        # Dyson
        self.g << inverse(inverse(self.g0) - self.sigma)
        # or self.g << self.g0 * inverse(1.0 - self.sigma * self.g0)

Solution of exercise 2
----------------------

This is the implementation of the DMFT loop. I added a couple of commands
to see the evolution of the spectral function.

In [ ]:
from pytriqs.plot.mpl_interface import *
from IPython.display import clear_output,display
%matplotlib inline

t = 1.0
U = 4.0
beta = 20
n_loops = 20

S = IPTSolver(beta = beta)
#S.g << inverse(inverse(SemiCircular(2*t))+U/2.0)
S.g << SemiCircular(2*t)
#S.g << (inverse(S.g0)-iOmega_n)/(-t**2)




fig = plt.figure(figsize=(12,8))

for i in range(n_loops):
    S.g0 << inverse( iOmega_n + (U/2.0 if i>8 else 0) - t**2 * S.g )
    S.solve(U = U,it=i)
    
    # Get real axis function with Pade approximants
    greal = GfReFreq(indices = [1], window = (-5.0,5.0))
    greal.set_from_pade(S.g, 100, 0.01)
    
    oplot(greal, RI='I', figure = fig, label = "Iteration = %i" % (i+1))
    clear_output()
#    display(fig)
    
#fig.clear()

Solution of exercise 3
----------------------

From the solution above it seems that 10 iterations are enough to converge the
DMFT loop. Now it is easy to repeat this for several values of $U$.

In [ ]:
t = 1.0
beta = 20
n_loops = 10

fig = figure(figsize=(6,6))
pn = 0

for U in arange(2.0, 7.5, 1.0):
    
    S = IPTSolver(beta = beta)
    S.g << SemiCircular(2*t)

    # DMFT
    for i in range(n_loops):
        S.g0 << inverse( iOmega_n - t**2 * S.g )
        S.solve(U)

    # Get the real-axis with Pade approximants
    greal = GfReFreq(indices = [1], window = (-8.0,8.0))
    greal.set_from_pade(S.g, 100, 0.01)
    
    clear_output()
    ax = fig.add_axes([0,1.-(pn+1)/6.,1,1./6.])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    oplot(greal, lw=3, RI='S', figure = fig, label = "U = %.2f" % U)
    plt.xlim(-8,8)
    plt.ylim(0,0.35)
    plt.ylabel("")
    display(fig)
    pn = pn+1
    
fig.clear()

Comparison with the litterature
-------------------------------

You can compare the result above with what can be found in the litterature
(review of Antoine Georges et al.)

<img src="files/mott.png" width=30%>